Machine Translation - English to Portuguese

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass
# TensorFlow ≥2.0 is required
import tensorflow as tf
assert tf.__version__ >= "2.0"
print(tf.__version__)

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)


# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")


import random

import tensorflow as tf
import string
import re

from tensorflow.keras import layers

2.9.1


In [68]:
# Criar a pasta pt-eng na diretoria de trabalho
# Colocar na pasta o ficheiro "por.txt"
# Se o ficheiro estiver armazenado noutro local, ajustar a primeira linha de código
# Adicionar "start" e "end" às strings target
import io
text_file = "./train.txt"
#text_file = "./dataset.txt"
with open(text_file, encoding='utf-8') as f:
    lines = f.read().split("\n")[:-1]

text_pairs = []

for line in lines:
    english, port = line.split("\t")
    port = "[start] " + port + " [end]"
    text_pairs.append((english, port))

print(text_pairs[-1])

('the bookmarked feed shows the posts that you have bookmarked. the bookmarked feed offers easy access to useful posts.', '[start] im feed mit lesezeichen werden alle post angezeigt, die sie mit einem lesezeichen versehen haben. der feed "mit lesezeichen" bietet ihnen schnellen zugriff auf nützliche posts. [end]')


In [69]:
# Visualizar uma amostra aleatória de 10 exemplos 

#for x in range(10):
#  print(x+1 , ": ", random.choice(text_pairs))

In [70]:
# Dividir o datastet em 3 conjuntos: treino, validação e teste

random.shuffle(text_pairs) #1. mistura todos os pairs

print("len(text_pairs) ->", len(text_pairs))

num_val_samples = int(0.15 * len(text_pairs))
print("15% for validation ->", num_val_samples)

num_train_samples = len(text_pairs) - 2 * num_val_samples
print("70% for train ->", num_train_samples)

train_pairs = text_pairs[:num_train_samples] #escolhe os primeiros 70% (shuffled) para treino
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples] #mais 15% para validação
test_pairs = text_pairs[num_train_samples + num_val_samples:] #mais 15 para teste

len(text_pairs) -> 100610
15% for validation -> 15091
70% for train -> 70428


In [71]:
# Qual a dimensão de cada conjunto?
print("treino: ",    len(train_pairs))
print("validação: ", len(val_pairs  ))
print("teste: ",     len(test_pairs ))

treino:  70428
validação:  15091
teste:  15091


In [72]:
# Modulo de pré-processament e vetorização do texto

strip_chars = string.punctuation # !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~

strip_chars = strip_chars.replace("[", "") # para não perder o [start] e [end]
strip_chars = strip_chars.replace("]", "") #

# É necessário criar uma standardização especial para as frases em português, de modo a não retirar os símbolos [ ]

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 10000    # O modelo apneas vai conhecer 15000 palavras
sequence_length = 10  # cada frase vai ter 20 palavrasg


source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length
)

target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)

print(train_pairs[0])
train_english_texts = [pair[0] for pair in train_pairs] 
print(train_english_texts[0])

train_pt_texts = [pair[1] for pair in train_pairs] 
print(train_english_texts[1])

source_vectorization.adapt(train_english_texts)
#source_vectorization.adapt([pair[0] for pair in text_pairs] )
target_vectorization.adapt(train_pt_texts)

('before uploading external data files, review the format requirements and examples of the .csv and metadata files in the analytics external data format reference.', '[start] prüfen sie vor dem hochladen von externen datendateien die formatanforderungen und beispiele der csv- und metadatendateien in der analytics external data format reference. [end]')
before uploading external data files, review the format requirements and examples of the .csv and metadata files in the analytics external data format reference.
horizontal bars to indicate scoring


In [73]:
#print(lines[-1])
print(train_english_texts[-1])

data size (bytes)


In [74]:
# Visualiz 

source_vectorization.get_vocabulary()  

['',
 '[UNK]',
 'the',
 'to',
 'a',
 'and',
 'in',
 'you',
 'for',
 'your',
 'of',
 'or',
 'that',
 'is',
 'can',
 'salesforce',
 'on',
 'if',
 'with',
 'from',
 'are',
 'field',
 'an',
 'page',
 'users',
 'user',
 'when',
 'this',
 'as',
 'create',
 'data',
 'select',
 'click',
 'use',
 'fields',
 'by',
 'custom',
 'record',
 'set',
 'list',
 'add',
 'all',
 'name',
 'it',
 'access',
 'lightning',
 'service',
 'records',
 'email',
 'have',
 'only',
 'view',
 'app',
 'be',
 'available',
 'object',
 'account',
 'type',
 'community',
 'see',
 'not',
 'new',
 'report',
 'using',
 'enter',
 'example',
 'settings',
 'edit',
 'up',
 'value',
 'then',
 'want',
 'related',
 'which',
 'default',
 'case',
 'information',
 'search',
 'values',
 'each',
 'time',
 'number',
 'permission',
 'one',
 'work',
 'any',
 'contact',
 'chatter',
 'org',
 'tab',
 'setup',
 'actions',
 'organization',
 'file',
 'has',
 'more',
 'enable',
 'change',
 'objects',
 'other',
 'their',
 'these',
 'they',
 'date',
 

In [75]:
# Visualizar o vocabulário em português

target_vectorization.get_vocabulary()

['',
 '[UNK]',
 '[start]',
 '[end]',
 'sie',
 'die',
 'der',
 'und',
 'in',
 'für',
 'auf',
 'den',
 'von',
 'können',
 'zu',
 'werden',
 'wenn',
 'mit',
 'das',
 'oder',
 'ein',
 'eine',
 'wird',
 'nicht',
 'aus',
 'um',
 'im',
 'des',
 'ist',
 'benutzer',
 'salesforce',
 'erstellen',
 'dem',
 'einen',
 'einer',
 'an',
 'klicken',
 'ihre',
 'sind',
 'angezeigt',
 'wählen',
 'über',
 'verwenden',
 'feld',
 'einem',
 'bei',
 'zum',
 'als',
 'diese',
 'ihrer',
 'sich',
 'eines',
 'alle',
 'nur',
 'nach',
 'geben',
 'zur',
 'anzeigen',
 'es',
 'wie',
 'dass',
 'kann',
 'bearbeiten',
 'hinzufügen',
 'unter',
 'organisation',
 'haben',
 'daten',
 'felder',
 'aktivieren',
 'lightning',
 'seite',
 'durch',
 'möchten',
 'beispielsweise',
 'dann',
 'benutzerdefinierte',
 'verfügbar',
 'verwendet',
 'informationen',
 'ihren',
 'datensätze',
 'keine',
 'ändern',
 'müssen',
 'auch',
 'wert',
 'löschen',
 'beim',
 'benutzerdefinierten',
 'dieser',
 'anwendung',
 'aktiviert',
 'erstellt',
 'zugriff'

In [76]:
glove50_file_path = "./glove.6B.50d.txt"

embeddings_index = {}

with open(glove50_file_path, encoding="utf-8") as gloveFile:
    for line in gloveFile:
        word, coefsAsString = line.split(maxsplit=1)
        coefs = np.fromstring(coefsAsString, "f", sep=" ")
        embeddings_index[word] = coefs

print(f"EN embeddings has {len(embeddings_index)} word vectors")


EN embeddings has 400000 word vectors


In [77]:
en_embeddings_dim = 50

en_vocabulary = source_vectorization.get_vocabulary() 

word_index = dict(zip(en_vocabulary, range(len(en_vocabulary)))) # setting an "id" to head word 

embedding_matrix = np.zeros((vocab_size, en_embeddings_dim)) 
print("embedding_matrix size:", embedding_matrix.shape)

for word, i in word_index.items():
    if(i < vocab_size):
        embedding_vector = embeddings_index.get(word)

    if(embedding_vector is not None):
        embedding_matrix[i] = embedding_vector

print(embedding_matrix[3:5])

en_embedding_layer = layers.Embedding(vocab_size, en_embeddings_dim, embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix))

embedding_matrix size: (10000, 50)
[[ 0.68046999 -0.039263    0.30186    -0.17792     0.42962     0.032246
  -0.41376001  0.13228001 -0.29846999 -0.085253    0.17117999  0.22419
  -0.10046    -0.43652999  0.33418     0.67846     0.057204   -0.34448001
  -0.42785001 -0.43274999  0.55962998  0.10032     0.18677001 -0.26853999
   0.037334   -2.09319997  0.22171    -0.39868     0.20912001 -0.55725002
   3.88260007  0.47466001 -0.95657998 -0.37788001  0.20869    -0.32752001
   0.12751     0.088359    0.16350999 -0.21634001 -0.094375    0.018324
   0.21048    -0.03088    -0.19722     0.082279   -0.09434    -0.073297
  -0.064699   -0.26043999]
 [ 0.21705     0.46515    -0.46757001  0.10082     1.01349998  0.74844998
  -0.53104001 -0.26256001  0.16812     0.13181999 -0.24909    -0.44185001
  -0.21739     0.51003999  0.13448    -0.43141001 -0.03123     0.20674001
  -0.78138    -0.20148    -0.097401    0.16088    -0.61835998 -0.18504
  -0.12461    -2.25259995 -0.22321001  0.5043      0.32257    

In [78]:
# Creating a data pipeline to feed into the translator model
# Utiliza o objeto tf.data

batch_size = 32

def format_dataset(eng, pt):
    eng = source_vectorization(eng)
    pt = target_vectorization(pt)
    return ({
        "english": eng,
        "portuguese": pt[:, :-1],
    }, pt[:, 1:])

def make_dataset(pairs):
    eng_texts, pt_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    pt_texts = list(pt_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, pt_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [79]:
# Visualizar o formato e alguns exemplos

for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['portuguese'].shape: {inputs['portuguese'].shape}")
    print(f"targets.shape: {targets.shape}")
    print(inputs['english'])


inputs['english'].shape: (32, 10)
inputs['portuguese'].shape: (32, 10)
targets.shape: (32, 10)
tf.Tensor(
[[   7   14  449   44    3 1809   47   18    2   44]
 [ 213   41 1384   16    2  204 1974    0    0    0]
 [  51    9   15  875  975  504    5  780   19    2]
 [  61  538  570   11   89    0    0    0    0    0]
 [   1  495    0    0    0    0    0    0    0    0]
 [   7   14  109   38  284  431   16  796   17    7]
 [  96 1051    0    0    0    0    0    0    0    0]
 [ 402  186 1100  186  179   38    3  475    6    2]
 [5450 3978    0    0    0    0    0    0    0    0]
 [   2  513  207   10    2   48   17  414   35    2]
 [ 600   14   29   91  163  600  137   40  141    3]
 [3160    3   48    0    0    0    0    0    0    0]
 [ 378  409   29    2  640  266    0    0    0    0]
 [ 354   10   80    2  220   94  363  835  199   22]
 [1708  332   20  492   26    0    0    0    0    0]
 [  39  367  144    5 2888 1965    6   45  112    0]
 [ 134  131    2  248  236   23  183    9  127

In [80]:
# Criação da classe que modela o Encoder 

# Na criação do objeto recebe 
# embed_dim: Dimensão da sequência de input 
# dense_dim: Número de nós da camada Dense
# num_heads: Número de attention heads

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

In [81]:
# Criação da classe que modela o Decoder 

# Na criação do objeto recebe 
# embed_dim: Dimensão da sequência de input 
# dense_dim: Número de nós da camada Dense
# num_heads: Número de attention heads

class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

In [82]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, wordEmbedding=None, **kwargs):
        super().__init__(**kwargs)

        if( wordEmbedding is not None ):
            self.token_embeddings = wordEmbedding
        else:
            self.token_embeddings = layers.Embedding(input_dim=input_dim, output_dim=output_dim)
        
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        
        self.sequence_length = sequence_length
        
        self.input_dim = input_dim
        
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

In [83]:
# The complete Transformer

import keras

# Settings 

#embed_dim = 256
embed_dim = 50
dense_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim, en_embedding_layer)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="portuguese")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)

transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])

In [85]:
 
transformer.fit(train_ds, epochs=5, validation_data=val_ds)

Epoch 1/5
2201/2201 [==============================] - 50s 22ms/step - loss: 3.5050 - accuracy: 0.4203 - val_loss: 3.3867 - val_accuracy: 0.4337
Epoch 2/5
2201/2201 [==============================] - 45s 20ms/step - loss: 3.4838 - accuracy: 0.4340 - val_loss: 3.3914 - val_accuracy: 0.4391
Epoch 3/5
2201/2201 [==============================] - 44s 20ms/step - loss: 3.4659 - accuracy: 0.4451 - val_loss: 3.3940 - val_accuracy: 0.4447
Epoch 4/5
2201/2201 [==============================] - 43s 20ms/step - loss: 3.4440 - accuracy: 0.4546 - val_loss: 3.3930 - val_accuracy: 0.4493
Epoch 5/5
2201/2201 [==============================] - 43s 19ms/step - loss: 3.4231 - accuracy: 0.4626 - val_loss: 3.3832 - val_accuracy: 0.4542


In [86]:
# Testar o desempenho do Transformer em frases do conjunto de teste

import numpy as np
pt_vocab = target_vectorization.get_vocabulary()
pt_index_lookup = dict(zip(range(len(pt_vocab)), pt_vocab))
max_decoded_sentence_length = 10

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = pt_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
event organizers
[start] [UNK] [end]
-
whether the requesttoken has been used before
[start] ob der [UNK] über die [UNK] [UNK] [end]
-
2,500
[start] [UNK] [end]
-
click new named credential, or click edit to modify an existing named credential.
[start] klicken sie auf neue [UNK] oder bearbeiten um die [UNK]
-
select access dialer monitoring.
[start] wählen sie [UNK] auf [UNK] [end]
-
new records display as compliant because they're not in violation.
[start] neue datensätze werden als [UNK] angezeigt wenn sie nicht die
-
authenticated website user
[start] [UNK] [end]
-
before you delete a quote, it’s a good idea to understand how deleting a quote affects quote-related pdfs and syncing with opportunities.
[start] bevor sie ein [UNK] löschen müssen sie sich in einem
-
pad_length is the number of total characters in the text string that will be returned.
[start] [UNK] ist die anzahl der [UNK] in den [UNK] in
-
icon in salesforce classic
[start] [UNK] in salesforce classic [end]
-
automat

In [924]:
# Testar o desempenho do transformer em frases introduzidas pelo utilizador
# Especificar 5 frases em inglês e verificar a qualidade da tradução

for _ in range(5):
    input_sentence = input()
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence), '\n')

-
how are you
[start] wie [UNK] sie [end] 

-

[start] [end] 

-

[start] [end] 



KeyboardInterrupt: Interrupted by user